In [1]:
import pandas as pd
import pickle
import copy
import cobra
import numpy as np
import matplotlib.pyplot as plt
from cobra.io import read_sbml_model,write_sbml_model
from cobra import Model, Reaction, Metabolite
from functools import reduce 
%matplotlib inline

In [2]:
mutant1 = read_sbml_model('mutant1.xml')

In [3]:
mutant1.medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


{'EX_ca2_e': 1000.0,
 'EX_cit_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_hdca_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_met__L_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0}

In [4]:
model2 = read_sbml_model('model2.xml')

In [5]:
model2.medium

{'EX_ca2_e': 1000.0,
 'EX_cit_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_met__L_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_no3_e': 1000.0,
 'EX_pi_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_zn2_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_hdca_e': 1000.0}

In [6]:
mutant1_reactions = {}
for i in mutant1.reactions:
    x=i.id
    y=i.build_reaction_string()
    mutant1_reactions[x] = y

In [7]:
model2_reactions = {}
for i in model2.reactions:
    x=i.id
    y=i.build_reaction_string()
    model2_reactions[x] = y

In [8]:
model2_unique = {}
for x in model2_reactions:
    if x in mutant1_reactions: continue
    else:
        y = model2_reactions[x]
        if x == 'BIOMASS_SCO': continue
        else:
            model2_unique[x] = y

In [9]:
model2.slim_optimize()

14.657320838057537

### Add sink reaction for acetyl coA

In [10]:
print("exchanges", len(mutant1.exchanges))
print("demands", len(mutant1.demands))
print("sinks", len(mutant1.sinks))

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


exchanges 343
demands 0
sinks 6


In [11]:
print("exchanges", len(model2.exchanges))
print("demands", len(model2.demands))
print("sinks", len(model2.sinks))

exchanges 158
demands 4
sinks 0


In [12]:
print("sinks", mutant1.sinks)

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


sinks [<Reaction sink_2ohph_c at 0x2580754dfa0>, <Reaction sink_4hba_c at 0x2580754daf0>, <Reaction sink_amob_c at 0x2580754d6d0>, <Reaction sink_hemeO_c at 0x25807536340>, <Reaction sink_lipopb_c at 0x2580754d910>, <Reaction sink_sheme_c at 0x2580754da30>]


In [13]:
mutant1.reactions.get_by_id('sink_2ohph_c')

Reaction identifier,sink_2ohph_c
Name,R_sink_2ohph_c
Memory address,0x2580754dfa0
Stoichiometry,2ohph_c --> 2-Octaprenyl-6-hydroxyphenol -->
GPR,
Lower bound,0.0
Upper bound,1000.0


In [14]:
reaction = Reaction('sink_accoa_c')
reaction.name = 'R_sink_accoa_c'
reaction.subsystem = 'Metabolite'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
reaction.gene_reaction_rule = ''

In [15]:
reaction.add_metabolites({mutant1.metabolites.accoa_c:-1})

In [16]:
mutant1.add_reactions([reaction])

### Set sink reaction as the objective

In [17]:
mutant1.objective='sink_accoa_c'

In [18]:
cobra.flux_analysis.pfba(mutant1)

,fluxes,reduced_costs
12DGR120tipp,0.000000,-2.0
12DGR140tipp,0.000000,-2.0
12DGR141tipp,0.000000,-2.0
12DGR160tipp,0.000000,-2.0
12DGR161tipp,0.000000,-2.0
...,...,...
MonB_Synth_c0,0.000000,26879.5
MonB_Transp_c0,0.000000,0.0
EX_MonensinA_e0,0.000000,-2.0
EX_MonensinB_e0,0.000000,0.0


### Adding Glucose to medium

In [19]:
medium = mutant1.medium
medium["EX_cit_e"] = 0.0
medium['EX_hdca_e'] = 1000
mutant1.medium = medium

Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.
Could not identify an external compartment by name and choosing one with the most boundary reactions. That might be complete nonsense or change suddenly. Consider renaming your compartments using `Model.compartments` to fix this.


In [20]:
solution = cobra.flux_analysis.pfba(mutant1)

### Find out why cells cannot use lipids. if you only use lipids can you produce acetyl coA.

In [21]:
#mutant1.objective='ATPM'

In [22]:
solution = cobra.flux_analysis.pfba(mutant1)

In [23]:
#model2.objective='ATPM'

In [24]:
solution1 = cobra.flux_analysis.pfba(model2)

In [25]:
print(mutant1.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []


In [26]:
mutant1.reactions.get_by_id("PDH").build_reaction_string(use_metabolite_names = True)

'Coenzyme A + Nicotinamide adenine dinucleotide + Pyruvate --> Acetyl-CoA + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced'

In [27]:
mutant1.reactions.get_by_id("PFL").build_reaction_string(use_metabolite_names = True)

'Coenzyme A + Pyruvate --> Acetyl-CoA + Formate'

In [28]:
mutant1.reactions.get_by_id("CS").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + H2O H2O + Oxaloacetate --> Citrate + Coenzyme A + H+'

In [29]:
print(model2.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent    Flux Reaction                                Definition
 16.62%   44.48  ACACT2r    accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
 16.62%   44.48  ACACT3r    accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
 16.68%   44.64  ACACT4r    accoa_c + occoa_c <=> 3odcoa_c + coa_c
 16.68%   44.64  ACACT5r  accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
 16.68%   44.64  ACACT6r accoa_c + ddcacoa_c <=> 3otdcoa_c + coa_c
 16.68%   44.64  ACACT7r   accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c
  0.03% 0.08019     PTAr         accoa_c + pi_c <=> actp_c + coa_c

Consuming Reactions
-------------------
Percent     Flux Reaction                                                 Definition
 91.29%   -244.3  ACACT1r                            2.0 accoa_c <=> aacoa_c + coa_c
  5.42%   -14.49   ACCOAC accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c
  1.35%   -3.624    CITMS              accoa_c + h2o_c + pyr_c <=> citm__R_c + coa_c
  1.91%   

In [30]:
 mutant1.metabolites.get_by_id('aacoa_c').summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [31]:
 mutant1.metabolites.get_by_id('coa_c').summary()

Percent,Flux,Reaction,Definition
Percent,Flux,Reaction,Definition


In [32]:
model2.reactions.get_by_id("ACACT2r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Butanoyl-CoA <=> 3-Oxohexanoyl-CoA + Coenzyme A'

In [33]:
model2.reactions.get_by_id("ACACT3r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Hexanoyl-CoA (n-C6:0CoA) <=> 3-Oxooctanoyl-CoA + Coenzyme A'

In [34]:
model2.reactions.get_by_id("ACACT4r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Octanoyl-CoA (n-C8:0CoA) <=> 3-Oxodecanoyl-CoA + Coenzyme A'

In [35]:
model2.reactions.get_by_id("ACACT5r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Decanoyl-CoA (n-C10:0CoA) <=> 3-Oxododecanoyl-CoA + Coenzyme A'

In [36]:
model2.reactions.get_by_id("ACACT6r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Dodecanoyl-CoA (n-C12:0CoA) <=> 3-Oxotetradecanoyl-CoA + Coenzyme A'

In [37]:
model2.reactions.get_by_id("ACACT7r").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Tetradecanoyl-CoA (n-C14:0CoA) <=> 3-Oxohexadecanoyl-CoA + Coenzyme A'

In [38]:
model2.reactions.get_by_id("PTAr").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Phosphate <=> Acetyl phosphate + Coenzyme A'

In [39]:
model2.reactions.get_by_id("ACACT1r").build_reaction_string(use_metabolite_names = True)

'2.0 Acetyl-CoA <=> Acetoacetyl-CoA + Coenzyme A'

In [40]:
model2.reactions.get_by_id("ACCOAC").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + ATP + Bicarbonate --> ADP + H+ + Malonyl-CoA + Phosphate'

In [41]:
model2.reactions.get_by_id("CITMS").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + H2O + Pyruvate <=> (R)-2-Methylmalate + Coenzyme A'

In [42]:
model2.reactions.get_by_id("GLYAT").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Glycine <=> L-2-Amino-3-oxobutanoate + Coenzyme A'

In [43]:
for x in mutant1.metabolites.get_by_id('2aobut_c').reactions:
    print(x.id)
    print(x.build_reaction_string(use_metabolite_names = False))
    print(solution[x.id])

GLYAT
accoa_c + gly_c <=> 2aobut_c + coa_c
0.0
ATHRDHr
athr__L_c + nadp_c <=> 2aobut_c + h_c + nadph_c
0.0
THRD
nad_c + thr__L_c --> 2aobut_c + h_c + nadh_c
0.0


In [44]:
li = ['3ohcoa_c','3oocoa_c','3odcoa_c','3oddcoa_c','3otdcoa_c','3ohdcoa_c']
for a in li:
    print(a)
    print(mutant1.metabolites.get_by_id(a).summary())

3ohcoa_c
3ohcoa_c
Formula: C27H40N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
3oocoa_c
3oocoa_c
Formula: C29H44N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
3odcoa_c
3odcoa_c
Formula: C31H48N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []
3oddcoa_c
3oddcoa_c
Formula: C33H52N7O18P3S

Producing Reactions
-------------------
Empty DataFrame
Columns: [Percent, Flux, Reaction, Definition]
Index: []

Consuming Reactions
-----------

In [45]:
li = ['3hhcoa_c','3ohcoa_c','3oocoa_c','3odcoa_c','3oddcoa_c','3otdcoa_c','3ohdcoa_c','3ohcoa_c','3hhdcoa_c','btcoa_c','hx2coa_c']
for a in li:
    print(a)
    for x in mutant1.metabolites.get_by_id(a).reactions:
        print(x.id)
        print(x.build_reaction_string(use_metabolite_names = False))
        print(solution[x.id])

3hhcoa_c
HACD2
3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c
0.0
ECOAH2
3hhcoa_c <=> h2o_c + hx2coa_c
0.0
HACD2i
3hhcoa_c + nad_c --> 3ohcoa_c + h_c + nadh_c
0.0
3ohcoa_c
ACACT2r
accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
0.0
HACD2
3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c
0.0
ACACT11
3ohcoa_c + coa_c --> accoa_c + btcoa_c
0.0
HACD2i
3hhcoa_c + nad_c --> 3ohcoa_c + h_c + nadh_c
0.0
3oocoa_c
ACACT3r
accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
0.0
HACD3i
3hocoa_c + nad_c --> 3oocoa_c + h_c + nadh_c
0.0
ACACT8
3oocoa_c + coa_c --> accoa_c + hxcoa_c
0.0
HACD3
3oocoa_c + h_c + nadh_c <=> 3hocoa_c + nad_c
0.0
3odcoa_c
HACD4i
3hdcoa_c + nad_c --> 3odcoa_c + h_c + nadh_c
0.0
ACACT4r
accoa_c + occoa_c <=> 3odcoa_c + coa_c
0.0
ACACT13
3odcoa_c + coa_c --> accoa_c + occoa_c
0.0
HACD4
3odcoa_c + h_c + nadh_c <=> 3hdcoa_c + nad_c
0.0
3oddcoa_c
HACD5i
3hddcoa_c + nad_c --> 3oddcoa_c + h_c + nadh_c
0.0
ACACT5r
accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
0.0
HACD5
3oddcoa_c + h_c + nadh_c <=> 3hddcoa_c + nad_c


In [46]:
li = ['3hhcoa_c','3ohcoa_c','3oocoa_c','3odcoa_c','3oddcoa_c','3otdcoa_c','3ohdcoa_c','3ohcoa_c','3hhdcoa_c','btcoa_c']
for a in li:
    print(a)
    print(model2.metabolites.get_by_id(a).summary())

3hhcoa_c
3hhcoa_c
Formula: C27H42N7O18P3S

Producing Reactions
-------------------
Percent  Flux Reaction                    Definition
100.00% 44.48   ECOAH2 3hhcoa_c <=> h2o_c + hx2coa_c

Consuming Reactions
-------------------
Percent   Flux Reaction                                   Definition
100.00% -44.48    HACD2 3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c
3ohcoa_c
3ohcoa_c
Formula: C27H40N7O18P3S

Producing Reactions
-------------------
Percent  Flux Reaction                                   Definition
100.00% 44.48    HACD2 3ohcoa_c + h_c + nadh_c <=> 3hhcoa_c + nad_c

Consuming Reactions
-------------------
Percent   Flux Reaction                             Definition
100.00% -44.48  ACACT2r accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
3oocoa_c
3oocoa_c
Formula: C29H44N7O18P3S

Producing Reactions
-------------------
Percent  Flux Reaction                                   Definition
100.00% 44.48    HACD3 3oocoa_c + h_c + nadh_c <=> 3hocoa_c + nad_c

Consuming Reactions
------

In [47]:
reactions_to_add = {}
ri = ['hdca_Transp_c0']
for x in model2_reactions:
    if x in ri:
        y = model2_reactions[x]
        reactions_to_add[x] = y

In [48]:
for x in reactions_to_add:
    reaction = model2.reactions.get_by_id(x)
    print(reaction)
    mutant1.add_reactions([reaction])
    print(mutant1.slim_optimize())

hdca_Transp_c0: hdca_e <=> hdca_c
12.056262558607244


In [49]:
mutant1.slim_optimize()

12.056262558606333

In [50]:
print(mutant1.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent  Flux Reaction                                                                      Definition
100.00% 533.2     FAO2 atp_c + 8.0 coa_c + 7.0 fad_c + 7.0 h2o_c + hdca_c + 7.0 nad_c --> 8.0 accoa...

Consuming Reactions
-------------------
Percent   Flux     Reaction                                         Definition
  0.75% -4.019      ACACT1r                    2.0 accoa_c <=> aacoa_c + coa_c
 94.72%   -505           CS    accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c
  2.26% -12.06         MALS accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c
  2.26% -12.06 sink_accoa_c                                       accoa_c --> 


In [51]:
mutant1.reactions.get_by_id("FAO2").build_reaction_string(use_metabolite_names = True)

'ATP C10H12N5O13P3 + 8.0 Coenzyme A + 7.0 Flavin adenine dinucleotide oxidized + 7.0 H2O H2O + Hexadecanoate (n-C16:0) + 7.0 Nicotinamide adenine dinucleotide --> 8.0 Acetyl-CoA + AMP C10H12N5O7P + 7.0 Flavin adenine dinucleotide reduced + 7.0 H+ + 7.0 Nicotinamide adenine dinucleotide - reduced + Diphosphate'

In [52]:
model2.reactions.get_by_id("MALS").build_reaction_string(use_metabolite_names = True)

'Acetyl-CoA + Glyoxylate + H2O --> Coenzyme A + H+ + L-Malate'

In [53]:
mutant1.objective='ATPM'

In [54]:
mutant1.slim_optimize()

375.00000000000006

In [55]:
print(mutant1.metabolites.accoa_c.summary())

accoa_c
Formula: C23H34N7O17P3S

Producing Reactions
-------------------
Percent Flux Reaction                                                                      Definition
100.00%  500     FAO2 atp_c + 8.0 coa_c + 7.0 fad_c + 7.0 h2o_c + hdca_c + 7.0 nad_c --> 8.0 accoa...

Consuming Reactions
-------------------
Percent Flux Reaction                                      Definition
100.00% -500       CS accoa_c + h2o_c + oaa_c --> cit_c + coa_c + h_c


In [57]:
mutant1.objective='ATPM'

In [58]:
mutant1.slim_optimize()

375.00000000000006

In [ ]:
#print(model2.metabolites.accoa_c.summary())